In [1]:
import pandas as pd
import numpy as np
from riotwatcher import LolWatcher, ApiError
from datetime import date
import ast
from pandas.io.json import json_normalize
import json
lol_watcher = LolWatcher('RGAPI-41c88fe1-f8d1-4ea4-83a7-9f5904bf3db7')
queue = "RANKED_SOLO_5x5"
my_region = 'BR1'

In [9]:
try:
    master_queue = lol_watcher.league.masters_by_queue(my_region,queue)
    grandmaster_queue = lol_watcher.league.grandmaster_by_queue(my_region,queue)
    challenger_queue = lol_watcher.league.grandmaster_by_queue(my_region,queue)
except ApiError as err:
    if err.response.status_code == 429:
        print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
        print('this retry-after is handled by default by the RiotWatcher library')
        print('future requests wait until the retry-after time passes')
    elif err.response.status_code == 404:
        print('Summoner with that ridiculous name not found.')
    else:
        raise

In [13]:
today = date.today().strftime("%d/%m/%Y")

In [29]:
master_queue_df = pd.DataFrame(master_queue['entries'])
master_queue_df['queue'] = 'master'
grandmaster_queue_df = pd.DataFrame(grandmaster_queue['entries'])
grandmaster_queue_df['queue'] = 'grandmaster'
challenger_queue_df = pd.DataFrame(challenger_queue['entries'])
challenger_queue_df['queue'] = 'challenger'

In [32]:
queues = pd.concat([grandmaster_queue_df,challenger_queue_df])

In [34]:
queues['date'] = today

In [35]:
queues.to_csv("queues_seed.csv",index=None)

In [30]:
print(f"Master User: {master_queue_df.summonerId.nunique()} GrandMaster Users : {grandmaster_queue_df.summonerId.nunique()} Challenger User: {challenger_queue_df.summonerId.nunique()}")

Master User: 2180 GrandMaster Users : 500 Challenger User: 500


In [2]:
df_queues = pd.read_csv("queues_seed.csv")

#### Uma vez que obtevemos as informações dos jogadores que fazem parte das queues desejadas, devemos obter as informações gerais de outros ids usados para captura das partidas

In [3]:
def get_id(row):
    summonner_id = row['summonerId']
    ids = lol_watcher.summoner.by_id(my_region, summonner_id)
    return ids
    

In [11]:
df_temp = pd.DataFrame()

In [12]:
df_temp['ids'] = df_queues.apply(get_id, axis = 1)

In [23]:
df_temp = df_temp['ids'].apply(pd.Series)

In [25]:
df_temp.to_csv("ids_data.csv")

### Uma vez que possuimos todos os ids necessarios, iremos buscar os dados de partida para cada player.

In [3]:
df_users = pd.read_csv("ids_data.csv")

In [5]:
df_queues.date = pd.to_datetime(df_queues.date)

In [6]:
import datetime
epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

In [7]:
df_matches = list()
up_date = int(unix_time_millis(df_queues.date[0]))

In [10]:
def get_matches(account_id, date_limit):
    up_date = int(unix_time_millis(df_queues.date[0]))
    matches_list = lol_watcher.match.matchlist_by_account(my_region,account_id)
    matches_list = pd.DataFrame(matches_list["matches"])
    matches_list = matches_list[matches_list.timestamp <= up_date].sort_values(by="timestamp")
    matches_list.drop(['platformId',"queue","season"],axis=1,inplace=True)
    return matches_list

In [11]:
ids_users = df_users.accountId.unique()
matches_data = list()
matches_detail_data = dict()
import time

In [13]:
count = 1
for id_u in ids_users:
    if (count % 100 ) == 0:
        print(count)
        count+=1
        pd.concat(matches_data).to_csv("matches_general_data_temp.csv")
        with open('matches_detailed_temp.json', 'w') as f:
            json.dump(matches_detail_data, f)
    try:
        match_list = get_matches(id_u, up_date)
        if not match_list.empty:
            match_list = match_list.head(10)
            match_list['accountId'] = id_u
            matches_data.append(match_list)
            games_ids = match_list.gameId.unique()
            for game_id in games_ids:
                matches_detail = lol_watcher.match.by_id(my_region, game_id)
                if matches_detail:
                    if game_id not in matches_detail_data:
                        matches_detail_data[str(game_id)] = matches_detail
                        time.sleep(0.05)     
    except ApiError as err:
        if err.response.status_code == 429:
            print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
            print('this retry-after is handled by default by the RiotWatcher library')
            print('future requests wait until the retry-after time passes')
        elif err.response.status_code == 404:
            print('Summoner with that ridiculous name not found.')
        else:
            print(err)
            pass

504 Server Error: Gateway Timeout for url: https://br1.api.riotgames.com/lol/match/v4/matches/2208934898
504 Server Error: Gateway Timeout for url: https://br1.api.riotgames.com/lol/match/v4/matchlists/by-account/2eiPSNoMI6KTvAh7UrqncoY_lfVRogV8IXZME9FYzq4HZDY
504 Server Error: Gateway Timeout for url: https://br1.api.riotgames.com/lol/match/v4/matches/2192459153
504 Server Error: Gateway Timeout for url: https://br1.api.riotgames.com/lol/match/v4/matches/2207443644
504 Server Error: Gateway Timeout for url: https://br1.api.riotgames.com/lol/match/v4/matches/2204510099
504 Server Error: Gateway Timeout for url: https://br1.api.riotgames.com/lol/match/v4/matches/2209185888
504 Server Error: Gateway Timeout for url: https://br1.api.riotgames.com/lol/match/v4/matchlists/by-account/j0j-LGJHKUvyrB_KFEnbZU1MYuztKopppd9NWHzONT6lQAYxaA3gG0n0
504 Server Error: Gateway Timeout for url: https://br1.api.riotgames.com/lol/match/v4/matches/2199479496
504 Server Error: Gateway Timeout for url: https:

In [ ]:
pd.concat(matches_data).to_csv("matches_general_data.csv")

with open('matches_detailed.json', 'w') as f:
    json.dump(matches_detail_data, f)

In [8]:
df_queues.groupby("summonerId")["queue"].count()

summonerId
-1gtBmSbZkwdH65S7sCM2BjcUOpjbgVUAuxtQREtgRjC5n4               2
-4hDcR3QcFs4Kcg9PABw2PfvLZGpwbc9bh2HmI8JxHJdZJU               2
-A2SIDC7zT_Sxl0Mi6TA_RnXTCFeJCRdlCLZ1B2jfEWhw2c               2
-KWhKQYh0TFOyfysqF1yUJraa-DT8dy5v8oXfypd4RIBcb-EoqQwBrxpiA    2
-Mzbp3qe6K1GRyQvnuTKxHPlRuMJka5xUqvfKEwgbGU2y7Y               2
                                                             ..
zWCznIbntNkJoWUM5t1EolznUmLF2r8Cny1kidkScpDOSw                2
zWfTEQuldVo2Ol3peYKT1e_U30dI5GL-Cubs5lHAfLVDOg                2
zuNseQefxJHpfcbqrh8jzua05yOaB15yWHhUlT6aziuqicM               2
zuZt-al5mEsHB0D66nZGlF8qT1JUKSkCKLIQWDUZ0q1CYQ                2
zz6Ab1ZNqenEdlFZRY125LKlB-FIxI67CDaSaJw7JEMNig                2
Name: queue, Length: 500, dtype: int64